first let see what recommendation engine do?
In basic words it help us to find content or provide suggestion which we are searching for.
Why recommendation engine is needed?
lets understand in term of psychology, our brain has 2 different system. system1 is automatic, fast, no effort is taken for getting job done. but for system2 the task which require attention, complex computation,its slow and effortfull.
our daily decision, beliefs,intuitions is generated by system1 (i.e actomatic) without effort.
If we start searching result without recommendation engine which will make our task more complex, effortful and attention is required which has to be done by system2.
Any thing which require effort, attention and complex to do, is most likely to get ignore by Humans begin
In basic words if the product is simple to use, effortless is most likely that users will reuse.
live example many times before reading you may think of egnoring and continue with code, which is the effortful task did by system2.

* The column listen_count denotes the no of times the song has been listened. Using this column, we’ll find the dataframe consisting of popular songs:

In [1]:
%matplotlib inline  
# To make data visualisations display in Jupyter Notebooks 
import numpy as np   # linear algebra
import pandas as pd  # Data processing, Input & Output load
import matplotlib.pyplot as plt # Visuvalization & plotting
import seaborn as sns # Also for Data visuvalization 

from sklearn.metrics.pairwise import cosine_similarity  # Compute cosine similarity between samples in X and Y.
from scipy import sparse  #  sparse matrix package for numeric data.
from scipy.sparse.linalg import svds # svd algorithm

import warnings   # To avoid warning messages in the code run
warnings.filterwarnings("ignore")
from math import sqrt
from scipy.spatial.distance import euclidean,cosine
import matplotlib.pyplot as plt
import squarify

In [2]:
import os
os.chdir(r"C:\Users\Asus\IMS_2term\Recommendation_music")

In [3]:
song = pd.read_csv('song_data.csv')
user = pd.read_excel('Book1.xlsx')

In [4]:
song.head()

,song_id,title,release,artist_name,year
0,SOQMMHC12AB0180CB8,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,2003
1,SOVFVAK12A8C1350D9,Tanssi vaan,Karkuteillä,Karkkiautomaatti,1995
2,SOGTUKN12AB017F4F1,No One Could Ever,Butter,Hudson Mohawke,2006
3,SOBNYVR12A8C13558C,Si Vos Querés,De Culo,Yerba Brava,2003
4,SOHSBXH12A8C13B0DF,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,0


In [5]:
user.head()

,user_id,song_id,listen_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0


In [6]:
song_df = pd.merge(user, song.drop_duplicates(['song_id']), on="song_id", how="left")

In [7]:
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,The Cove,Thicker Than Water,Jack Johnson,0.0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,Constellations,In Between Dreams,Jack Johnson,2005.0
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999.0


In [8]:
len(song_df)

115706

In [9]:
#song_crosstab.to_csv("book4.csv")

In [10]:
song_df.title.is_unique

False

In [11]:
song_df.release.is_unique

False

In [12]:
song_df.user_id.is_unique

False

In [13]:
song_df.shape

(115706, 7)

* 120707 - 115706 = 5001
* There are 5,001 song_id duplicate, we can't use duplicate song_id because we don't have user_id to work on.

In [14]:
# creating popular music recommendation and similar music recommendation (personel music recommendation)

In [15]:
n_user = song_df.user_id.unique().shape[0]
n_song = song_df.song_id.unique().shape[0]
print(n_user,n_song)

4448 9973


In [16]:
song_df.describe()

,listen_count,year
count,115705.000000,115705.000000
mean,3.002506,1631.711897
std,6.774812,776.317869
min,1.000000,0.000000
25%,1.000000,1984.000000
50%,1.000000,2002.000000
75%,3.000000,2007.000000
max,796.000000,2010.000000


In [17]:
song_df['song'] = song_df['title'].map(str) + " - " + song_df['artist_name']

In [18]:
song_df.head()

,user_id,song_id,listen_count,title,release,artist_name,year,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1.0,The Cove,Thicker Than Water,Jack Johnson,0.0,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2.0,Entre Dos Aguas,Flamenco Para Niños,Paco De Lucia,1976.0,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1.0,Stronger,Graduation,Kanye West,2007.0,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1.0,Constellations,In Between Dreams,Jack Johnson,2005.0,Constellations - Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1.0,Learn To Fly,There Is Nothing Left To Lose,Foo Fighters,1999.0,Learn To Fly - Foo Fighters


In [19]:
song_grouped = song_df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()

In [20]:
song_grouped.head()

,song,listen_count
0,#!*@ You Tonight [Featuring R. Kelly] (Explici...,2
1,#40 - DAVE MATTHEWS BAND,13
2,& Down - Boys Noize,28
3,' Cello Song - Nick Drake,7
4,'97 Bonnie & Clyde - Eminem,11


In [21]:
song_stats = pd.DataFrame(song_df.groupby('song')['listen_count'].mean())
song_stats.sort_values(by = ['listen_count'],ascending=False).head()

,listen_count
song,
The Good Fight - Dashboard Confessional,105.000000
Paradise & Dreams - Darren Styles,81.545455
Cold Blooded (Acid Cleanse) - The fFormula,62.250000
Move Shake Drop Remix - DJ Laz,58.000000
Waiting For A Dream - Rufus Wainwright,50.600000


In [22]:
song_stats['Count'] = pd.DataFrame(song_df.groupby('song')['listen_count'].count())
song_stats.sort_values(by =['Count'], ascending=False).head()

,listen_count,Count
song,,
Sehr kosmisch - Harmonia,3.587525,497
Undo - Björk,7.172986,422
Dog Days Are Over (Radio Edit) - Florence + The Machine,3.882212,416
Secrets - OneRepublic,3.775623,361
Revelry - Kings Of Leon,7.717087,357


In [23]:
song_stats.shape

(9925, 2)

## Finding the Similarities between movies
* we will use the correlation between the listen_count of songs as the similarity metric.
* To see correlation we will create pivot table

In [24]:
song_crosstab = song_df.pivot_table(index='user_id', columns='song', values='listen_count')
song_crosstab.head()

song,#!*@ You Tonight [Featuring R. Kelly] (Explicit Album Version) - The Notorious B.I.G.,#40 - DAVE MATTHEWS BAND,& Down - Boys Noize,' Cello Song - Nick Drake,'97 Bonnie & Clyde - Eminem,'Round Midnight - Amy Winehouse,'Round Midnight - Miles Davis,'Til We Die (Album Version) - Slipknot,'Till I Collapse - Eminem / Nate Dogg,('Til) I Kissed You - Everly Brothers,...,sillyworld (Album Version) - Stone Sour,sleep_ eat food_ have visions - Four Tet,smile around the face - Four Tet,sun drums and soil - Four Tet,teachme (Album Version) - Musiq Soulchild,the Love Song - K-OS,you were there with me - Four Tet,¡Viva La Gloria! (Album Version) - Green Day,¿Lo Ves? [Piano Y Voz] - Alejandro Sanz,Época - Gotan Project
user_id,,,,,,,,,,,,,,,,,,,,,
0007c0e74728ca9ef0fe4eb7f75732e8026a278b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000ebc858861aca26bac9b49f650ed424cf882fc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00342a0cdf56a45465f09a39040a5bc25b7d0046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0039bd8483d578997718cdc0bf6c7c88b679f488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00498f4bab2bfeb17680113c7d9525ad5b0ad401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
song_crosstab['Sehr kosmisch - Harmonia']

user_id
0007c0e74728ca9ef0fe4eb7f75732e8026a278b   NaN
000ebc858861aca26bac9b49f650ed424cf882fc   NaN
00342a0cdf56a45465f09a39040a5bc25b7d0046   NaN
0039bd8483d578997718cdc0bf6c7c88b679f488   NaN
00498f4bab2bfeb17680113c7d9525ad5b0ad401   NaN
                                            ..
ffc7fc1eebeaf5cd0d6fff16c7b38726638a19c6   NaN
ffd782fcce37e276e6ab846dcc36a53c42d563fe   NaN
ffdaab327f2fc6b9fa01a4e3e7f41fdd0e468046   NaN
ffe50146303f1b12ea8254450b95580b1b99a5c4   NaN
fffce9c1537fbc350ea68823d956eaa8f5236dbe   NaN
Name: Sehr kosmisch - Harmonia, Length: 4447, dtype: float64

## Correlation similarity
* we can find the correlation between the listen_count for the given song and all the other songs using corrwith() function as shown
* It use pearson correlation value where the data is match

In [26]:
Similarity = song_crosstab.corrwith(song_crosstab['Sehr kosmisch - Harmonia'])
Similarity

song
#!*@ You Tonight [Featuring R. Kelly] (Explicit Album Version) - The Notorious B.I.G.    NaN
#40 - DAVE MATTHEWS BAND                                                                 1.0
& Down - Boys Noize                                                                      NaN
' Cello Song - Nick Drake                                                                NaN
'97 Bonnie & Clyde - Eminem                                                              NaN
                                                                                        ... 
the Love Song - K-OS                                                                     NaN
you were there with me - Four Tet                                                        NaN
¡Viva La Gloria! (Album Version) - Green Day                                             NaN
¿Lo Ves? [Piano Y Voz] - Alejandro Sanz                                                  NaN
Época - Gotan Project                                            

In [27]:
Similarity = song_crosstab.corrwith(song_crosstab['Undo - Björk'])
Similarity

song
#!*@ You Tonight [Featuring R. Kelly] (Explicit Album Version) - The Notorious B.I.G.         NaN
#40 - DAVE MATTHEWS BAND                                                                      NaN
& Down - Boys Noize                                                                     -0.505291
' Cello Song - Nick Drake                                                                     NaN
'97 Bonnie & Clyde - Eminem                                                                   NaN
                                                                                           ...   
the Love Song - K-OS                                                                          NaN
you were there with me - Four Tet                                                             NaN
¡Viva La Gloria! (Album Version) - Green Day                                                  NaN
¿Lo Ves? [Piano Y Voz] - Alejandro Sanz                                                       NaN
Época - Gotan P

In [28]:
corr_similar = pd.DataFrame(Similarity, columns=['Correlation'])
corr_similar.sort_values(['Correlation'], ascending= False).head(10)

,Correlation
song,
One More Sad Song - The All-American Rejects,1.0
Not Big - Lily Allen,1.0
Can't Have You - Jonas Brothers,1.0
Campus (Album) - Vampire Weekend,1.0
The Wind Blows - The All-American Rejects,1.0
Flume - Bon Iver,1.0
I'm In The House - Steve Aoki Feat. [[[Zuper Blahq]]],1.0
Calendar girl - Stars,1.0
These Boots - Eric Church,1.0


* We will add the count of listen_count also to see why many song are exactly correlating for the single songs

In [29]:
corr_similar_num_of_count = corr_similar.join(song_stats['Count'])
corr_similar_num_of_count.sort_values(['Correlation'], ascending= False).head(10)

,Correlation,Count
song,,
One More Sad Song - The All-American Rejects,1.0,21
Not Big - Lily Allen,1.0,12
Can't Have You - Jonas Brothers,1.0,11
Campus (Album) - Vampire Weekend,1.0,23
The Wind Blows - The All-American Rejects,1.0,9
Flume - Bon Iver,1.0,37
I'm In The House - Steve Aoki Feat. [[[Zuper Blahq]]],1.0,6
Calendar girl - Stars,1.0,6
These Boots - Eric Church,1.0,9


In [30]:
corr_similar_num_of_count[corr_similar_num_of_count ['Count']>50].sort_values('Correlation', ascending=False).head()

,Correlation,Count
song,,
Undo - Björk,1.000000,422
When I Grow Up - The Pussycat Dolls,0.984301,53
I Wanna Be Sedated (Remastered Album Version ) - Ramones,0.979149,66
LoveStoned/I Think She Knows - Justin Timberlake,0.971910,62
My Immortal (Album Version) - Evanescence,0.969193,57


## creating the user defined function so that we can pass song and get recommendation

In [31]:
def get_recommendations(song):
    # Get the song_df of the song column that matches the song
    song_df = song_crosstab[song]

    # Get the  similarity corrilated  scores of all songs with that song
    sim_scores = song_crosstab.corrwith(song_df)

    # Sort the song based on the similarity scores
    corr_title = pd.DataFrame(sim_scores, columns=['Correlation'])
    
    # Removing na values 
    corr_title.dropna(inplace=True)
    
    corr_title = corr_title.join(song_stats['Count'])
    
    # Return the top 10 most similar songs
    return corr_title[corr_title ['Count']>50].sort_values('Correlation', ascending=False).head()

In [32]:
get_recommendations('When I Grow Up - The Pussycat Dolls')

,Correlation,Count
song,,
Behind The Sea [Live In Chicago] - Panic At The Disco,1.0,89
Rabbit Heart (Raise It Up) - Florence + The Machine,1.0,80
Tive Sim - Cartola,1.0,280
When I Grow Up - The Pussycat Dolls,1.0,53
Don't Start Me Talkin' - Sonny Boy Williamson,1.0,56
